In [3]:
!pip3 install langchain

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 222 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.6 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 23.4 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 26.9 MB/s eta 0:00:01
  Created wheel for SQLAlchemy: filename=SQLAlchemy-1.4.46-cp39-cp39-macosx_10_9_universal2.whl size=1578667 sha256=9991d70fde083b993d7fe1fd61fca33a279e921f1b8296b02037e24b8cac1097
  Stored in directory: /Users/nmehta/Library/Caches/pip/wheels/3c/99/65/57cf5a0ec6e7f3b803a68d31694501e168997e03e80adc903d
Successfully built SQLAlchemy
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not w

In [1]:
from langchain.docstore.document import Document

In [2]:
example_doc_1 = """
Peter and Elizabeth took a taxi to attend the night party in the city. While in the party, Elizabeth collapsed and was rushed to the hospital.
Since she was diagnosed with a brain injury, the doctor told Peter to stay besides her until she gets well.
Therefore, Peter stayed with her at the hospital for 3 days without leaving.
"""

docs = [
    Document(
        page_content=example_doc_1,
    )
]

In [4]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain, SagemakerEndpoint
from langchain.chains.question_answering import load_qa_chain
import json

query = """How long was Elizabeth hospitalized?
"""

prompt_template = """Use the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

def model_input_transform_fn(prompt, model_kwargs):
    parameter_payload = {"inputs": prompt, "parameters": model_kwargs}
    return json.dumps(parameter_payload).encode("utf-8") 

chain = load_qa_chain(llm=SagemakerEndpoint(
                    endpoint_name="my-sagemaker-model-endpoint", 
                    credentials_profile_name="credentials-profile-name", 
                    region_name="us-west-2", 
                    model_kwargs={"temperature":1e-10},
                    content_type="application/json", 
                    model_input_transform_fn=model_input_transform_fn), 
                prompt=PROMPT)          

chain({"input_documents": docs, "question": query}, return_only_outputs=True)


{'output_text': '3 days'}